# Fase 2: Cálculo de Índices Espectrales
## Detección de Cambios Urbanos - Peñaflor

**Proyecto:** Laboratorio de Desarrollo de Aplicaciones Geoinformáticas  
**Universidad:** Universidad de Santiago de Chile  
**Zona de estudio:** Comuna de Peñaflor, Región Metropolitana

---

## Objetivo

Calcular los índices espectrales requeridos para detectar cambios urbanos y de vegetación:

| Índice | Fórmula | Detecta |
|--------|---------|---------|
| **NDVI** | $\frac{NIR - Red}{NIR + Red}$ | Vegetación |
| **NDBI** | $\frac{SWIR - NIR}{SWIR + NIR}$ | Áreas construidas |
| **NDWI** | $\frac{Green - NIR}{Green + NIR}$ | Cuerpos de agua |
| **BSI** | $\frac{(SWIR + Red) - (NIR + Blue)}{(SWIR + Red) + (NIR + Blue)}$ | Suelo desnudo |

**Bandas Sentinel-2 utilizadas:**
- B2 (Blue, 490nm) - 10m resolución
- B3 (Green, 560nm) - 10m resolución  
- B4 (Red, 665nm) - 10m resolución
- B8 (NIR, 842nm) - 10m resolución
- B11 (SWIR1, 1610nm) - 20m resolución

## 1. Configuración del Entorno

Importamos las bibliotecas necesarias para el procesamiento de imágenes satelitales.

In [ ]:
# Importar bibliotecas
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurar tamaño de figuras
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 10

print("✅ Bibliotecas cargadas correctamente")
print(f"📅 Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f"🐍 Versiones:")
print(f"   - rasterio: {rasterio.__version__}")
print(f"   - numpy: {np.__version__}")
print(f"   - pandas: {pd.__version__}")

## 2. Definición de Rutas y Parámetros

Configuramos las rutas de entrada/salida y los parámetros de procesamiento.

In [ ]:
# Rutas de trabajo
BASE_DIR = Path.cwd().parent
INPUT_DIR = BASE_DIR / 'data' / 'raw'
OUTPUT_DIR = BASE_DIR / 'data' / 'processed'

# Crear carpeta de salida si no existe
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Años a procesar
YEARS = [2018, 2020, 2022, 2024]

# Parámetro para evitar división por cero
EPSILON = 1e-10

print("Configuración:")
print(f"  Directorio de entrada: {INPUT_DIR}")
print(f"  Directorio de salida: {OUTPUT_DIR}")
print(f"  Años a procesar: {YEARS}")
print(f"  Epsilon: {EPSILON}")

## 3. Funciones de Cálculo de Índices

Implementamos las funciones para calcular los cuatro índices espectrales.

In [ ]:
def calcular_indices(input_path, output_path, eps=1e-10):
    """
    Calcula índices espectrales NDVI, NDBI, NDWI y BSI a partir de imagen Sentinel-2.
    
    Parámetros:
    -----------
    input_path : Path o str
        Ruta al archivo GeoTIFF de entrada (imagen Sentinel-2)
    output_path : Path o str
        Ruta donde guardar los índices calculados
    eps : float
        Valor pequeño para evitar división por cero (default: 1e-10)
        
    Retorna:
    --------
    dict : Diccionario con estadísticas de cada índice
    """
    
    print(f"\n{'='*60}")
    print(f"Procesando: {input_path.name}")
    print(f"{'='*60}")
    
    # Abrir imagen Sentinel-2
    with rasterio.open(input_path) as src:
        # Leer bandas (B2=Blue, B3=Green, B4=Red, B8=NIR, B11=SWIR1, B12=SWIR2)
        blue = src.read(1).astype(np.float32)
        green = src.read(2).astype(np.float32)
        red = src.read(3).astype(np.float32)
        nir = src.read(4).astype(np.float32)
        swir1 = src.read(5).astype(np.float32)
        swir2 = src.read(6).astype(np.float32)
        
        # Obtener metadatos para el archivo de salida
        profile = src.profile.copy()
        
        # Detectar si los valores están en Digital Numbers (0-10000) o ya escalados (0-1)
        max_val = max(np.nanmax(red), np.nanmax(nir), np.nanmax(swir1))
        if max_val > 10:
            print(f"  Valores detectados como DN (max={max_val:.0f}). Escalando a reflectancia...")
            blue = blue / 10000.0
            green = green / 10000.0
            red = red / 10000.0
            nir = nir / 10000.0
            swir1 = swir1 / 10000.0
            swir2 = swir2 / 10000.0
        else:
            print(f"  Valores ya escalados (max={max_val:.4f})")
    
    # 1. NDVI - Normalized Difference Vegetation Index
    # Mide la densidad de vegetación verde
    ndvi = (nir - red) / (nir + red + eps)
    
    # 2. NDBI - Normalized Difference Built-up Index
    # Resalta áreas urbanas construidas
    ndbi = (swir1 - nir) / (swir1 + nir + eps)
    
    # 3. NDWI - Normalized Difference Water Index
    # Identifica cuerpos de agua
    ndwi = (green - nir) / (green + nir + eps)
    
    # 4. BSI - Bare Soil Index
    # Identifica suelo desnudo
    bsi = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue) + eps)
    
    # Crear máscara de valores válidos
    valid_mask = ~(np.isnan(ndvi) | np.isnan(ndbi) | np.isnan(ndwi) | np.isnan(bsi))
    
    # Reemplazar NaN con -9999 (valor nodata estándar)
    ndvi[~valid_mask] = -9999
    ndbi[~valid_mask] = -9999
    ndwi[~valid_mask] = -9999
    bsi[~valid_mask] = -9999
    
    # Calcular estadísticas (solo píxeles válidos)
    stats = {}
    for name, arr in [('NDVI', ndvi), ('NDBI', ndbi), ('NDWI', ndwi), ('BSI', bsi)]:
        valid_data = arr[valid_mask]
        if len(valid_data) > 0:
            stats[name] = {
                'mean': float(np.mean(valid_data)),
                'std': float(np.std(valid_data)),
                'min': float(np.min(valid_data)),
                'max': float(np.max(valid_data)),
                'median': float(np.median(valid_data))
            }
            print(f"\n  {name}:")
            print(f"    Media: {stats[name]['mean']:7.4f}")
            print(f"    Desv. Est.: {stats[name]['std']:7.4f}")
            print(f"    Mín-Máx: [{stats[name]['min']:7.4f}, {stats[name]['max']:7.4f}]")
            print(f"    Mediana: {stats[name]['median']:7.4f}")
        else:
            stats[name] = None
    
    # Configurar perfil para archivo de salida (4 bandas)
    profile.update({
        'count': 4,
        'dtype': 'float32',
        'nodata': -9999
    })
    
    # Guardar índices en un archivo multi-banda
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(ndvi, 1)
        dst.write(ndbi, 2)
        dst.write(ndwi, 3)
        dst.write(bsi, 4)
        
        # Agregar nombres de banda como metadata
        dst.set_band_description(1, 'NDVI')
        dst.set_band_description(2, 'NDBI')
        dst.set_band_description(3, 'NDWI')
        dst.set_band_description(4, 'BSI')
    
    print(f"\n✓ Índices guardados en: {output_path.name}")
    
    return stats

# Probar la función con un mensaje
print("✓ Función calcular_indices() definida correctamente")

## 4. Procesamiento por Lotes

Procesamos todas las imágenes Sentinel-2 disponibles.

In [ ]:
# Diccionario para almacenar estadísticas de todos los años
all_stats = {}

# Procesar cada año
for year in YEARS:
    input_file = INPUT_DIR / f'sentinel2_{year}.tif'
    output_file = OUTPUT_DIR / f'indices_{year}.tif'
    
    if input_file.exists():
        stats = calcular_indices(input_file, output_file, eps=EPSILON)
        all_stats[year] = stats
    else:
        print(f"\n⚠ ADVERTENCIA: No se encontró {input_file.name}")

print(f"\n{'='*60}")
print(f"RESUMEN: Se procesaron {len(all_stats)} imágenes")
print(f"{'='*60}")

## 5. Análisis Estadístico

Creamos una tabla comparativa con las estadísticas de todos los años.

In [ ]:
# Crear DataFrame con estadísticas
rows = []
for year, stats in all_stats.items():
    for index_name in ['NDVI', 'NDBI', 'NDWI', 'BSI']:
        if stats[index_name] is not None:
            row = {
                'Año': year,
                'Índice': index_name,
                'Media': stats[index_name]['mean'],
                'Desv.Est.': stats[index_name]['std'],
                'Mínimo': stats[index_name]['min'],
                'Máximo': stats[index_name]['max'],
                'Mediana': stats[index_name]['median']
            }
            rows.append(row)

df_stats = pd.DataFrame(rows)

# Mostrar tabla
print("\nTabla de Estadísticas Completa:")
print("="*90)
display(df_stats)

# Guardar tabla a CSV
csv_path = OUTPUT_DIR / 'estadisticas_indices.csv'
df_stats.to_csv(csv_path, index=False)
print(f"\n✓ Estadísticas guardadas en: {csv_path.name}")

## 6. Visualización Comparativa

Creamos mapas comparativos de los cuatro índices espectrales para todos los años.

In [ ]:
# Configuración de colormaps para cada índice
colormaps = {
    'NDVI': 'RdYlGn',    # Rojo-Amarillo-Verde (vegetación)
    'NDBI': 'RdPu',      # Rojo-Púrpura (áreas urbanas)
    'NDWI': 'Blues',     # Azules (agua)
    'BSI': 'YlOrBr'      # Amarillo-Naranja-Café (suelo desnudo)
}

index_names = ['NDVI', 'NDBI', 'NDWI', 'BSI']

# Crear figura con subplots (4 índices × 4 años = 16 subplots)
fig, axes = plt.subplots(4, 4, figsize=(20, 20))
fig.suptitle('Índices Espectrales - Peñaflor (2018-2024)', fontsize=20, fontweight='bold', y=0.995)

for i, index_name in enumerate(index_names):
    for j, year in enumerate(YEARS):
        ax = axes[i, j]
        
        # Leer banda correspondiente al índice
        indices_file = OUTPUT_DIR / f'indices_{year}.tif'
        
        if indices_file.exists():
            with rasterio.open(indices_file) as src:
                # NDVI=banda1, NDBI=banda2, NDWI=banda3, BSI=banda4
                band_num = i + 1
                data = src.read(band_num)
                
                # Aplicar máscara de nodata
                data_masked = np.ma.masked_equal(data, -9999)
                
                # Calcular percentiles para mejorar visualización (evitar outliers)
                vmin, vmax = np.percentile(data_masked.compressed(), [2, 98])
                
                # Mostrar imagen
                im = ax.imshow(data_masked, cmap=colormaps[index_name], vmin=vmin, vmax=vmax)
                
                # Agregar colorbar
                cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
                cbar.ax.tick_params(labelsize=8)
        else:
            ax.text(0.5, 0.5, 'Datos no disponibles', 
                   ha='center', va='center', transform=ax.transAxes, fontsize=10)
        
        # Títulos
        if i == 0:
            ax.set_title(f'{year}', fontsize=14, fontweight='bold')
        if j == 0:
            ax.set_ylabel(index_name, fontsize=14, fontweight='bold')
        
        # Quitar ejes
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'comparacion_indices.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Mapa comparativo generado")

## 7. Análisis de Distribución

Visualizamos la distribución de valores mediante histogramas y curvas de densidad.

In [ ]:
# Crear figura con histogramas (4 índices)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Distribución de Índices Espectrales (2018-2024)', fontsize=18, fontweight='bold')

axes = axes.flatten()

for idx, index_name in enumerate(index_names):
    ax = axes[idx]
    
    # Recopilar datos de todos los años para este índice
    for year in YEARS:
        indices_file = OUTPUT_DIR / f'indices_{year}.tif'
        
        if indices_file.exists():
            with rasterio.open(indices_file) as src:
                # Leer banda correspondiente
                band_num = idx + 1
                data = src.read(band_num)
                
                # Filtrar nodata y obtener valores válidos
                valid_data = data[data != -9999]
                
                # Histograma + KDE
                ax.hist(valid_data, bins=100, alpha=0.4, label=f'{year}', density=True)
                
                # Agregar curva de densidad (KDE)
                from scipy.stats import gaussian_kde
                if len(valid_data) > 100:  # Solo si hay suficientes datos
                    kde = gaussian_kde(valid_data)
                    x_range = np.linspace(valid_data.min(), valid_data.max(), 200)
                    ax.plot(x_range, kde(x_range), linewidth=2, label=f'{year} KDE')
    
    ax.set_xlabel('Valor del Índice', fontsize=12)
    ax.set_ylabel('Densidad de Probabilidad', fontsize=12)
    ax.set_title(f'{index_name}', fontsize=14, fontweight='bold')
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'histogramas_indices.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Histogramas generados")

## 8. Análisis de Tendencias Temporales

Evaluamos cómo han cambiado los índices entre 2018 y 2024.

In [ ]:
# Crear tabla de tendencias
print("\nAnálisis de Tendencias (2018 → 2024)")
print("="*70)

trend_data = []

for index_name in index_names:
    if 2018 in all_stats and 2024 in all_stats:
        if all_stats[2018][index_name] and all_stats[2024][index_name]:
            valor_2018 = all_stats[2018][index_name]['mean']
            valor_2024 = all_stats[2024][index_name]['mean']
            cambio_abs = valor_2024 - valor_2018
            cambio_rel = (cambio_abs / abs(valor_2018)) * 100 if valor_2018 != 0 else 0
            
            trend_data.append({
                'Índice': index_name,
                '2018 (media)': f"{valor_2018:.4f}",
                '2024 (media)': f"{valor_2024:.4f}",
                'Cambio Absoluto': f"{cambio_abs:+.4f}",
                'Cambio Relativo (%)': f"{cambio_rel:+.2f}"
            })

df_trends = pd.DataFrame(trend_data)
display(df_trends)

# Interpretación
print("\n📊 Interpretación de Tendencias:")
print("-" * 70)

for _, row in df_trends.iterrows():
    index_name = row['Índice']
    cambio = float(row['Cambio Absoluto'])
    
    if index_name == 'NDVI':
        if cambio < 0:
            print(f"  • {index_name}: ⚠️  Disminución de vegetación (posible urbanización)")
        else:
            print(f"  • {index_name}: ✅ Aumento de vegetación")
    
    elif index_name == 'NDBI':
        if cambio > 0:
            print(f"  • {index_name}: 🏗️  Aumento de áreas construidas (expansión urbana)")
        else:
            print(f"  • {index_name}: Disminución de áreas construidas")
    
    elif index_name == 'NDWI':
        if cambio < 0:
            print(f"  • {index_name}: ⚠️  Reducción de cuerpos de agua")
        else:
            print(f"  • {index_name}: ✅ Incremento de agua superficial")
    
    elif index_name == 'BSI':
        if cambio > 0:
            print(f"  • {index_name}: ⚠️  Aumento de suelo desnudo")
        else:
            print(f"  • {index_name}: Reducción de suelo desnudo")

print("\n" + "="*70)

## 9. Gráficos de Evolución Temporal

Visualizamos la evolución de cada índice a lo largo del tiempo.

In [ ]:
# Crear gráfico de líneas con la evolución temporal
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Evolución Temporal de Índices Espectrales - Peñaflor', fontsize=16, fontweight='bold')

axes = axes.flatten()

for idx, index_name in enumerate(index_names):
    ax = axes[idx]
    
    # Extraer valores medios para cada año
    years_list = []
    means = []
    stds = []
    
    for year in YEARS:
        if year in all_stats and all_stats[year][index_name]:
            years_list.append(year)
            means.append(all_stats[year][index_name]['mean'])
            stds.append(all_stats[year][index_name]['std'])
    
    # Graficar línea de tendencia con área de desviación estándar
    if len(years_list) > 0:
        means_arr = np.array(means)
        stds_arr = np.array(stds)
        
        # Línea principal
        ax.plot(years_list, means, marker='o', linewidth=2.5, markersize=10, label='Media')
        
        # Banda de confianza (± 1 desviación estándar)
        ax.fill_between(years_list, means_arr - stds_arr, means_arr + stds_arr, 
                        alpha=0.3, label='± 1 Desv.Est.')
    
    ax.set_xlabel('Año', fontsize=12, fontweight='bold')
    ax.set_ylabel(f'Valor de {index_name}', fontsize=12, fontweight='bold')
    ax.set_title(index_name, fontsize=14, fontweight='bold')
    ax.legend(loc='best', fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.set_xticks(YEARS)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'evolucion_temporal.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Gráficos de evolución temporal generados")

## 10. Conclusiones

**Resumen de la Fase 2:**

✅ **Procesamiento Completado:**
- Se calcularon 4 índices espectrales (NDVI, NDBI, NDWI, BSI) para los años 2018, 2020, 2022 y 2024
- Se generaron archivos GeoTIFF multi-banda con los índices calculados
- Se crearon visualizaciones comparativas y análisis estadísticos

📊 **Próximos Pasos (Fase 3):**
- Detección de cambios mediante análisis de diferencias temporales
- Identificación de zonas con mayor urbanización (ΔNDBI positivo, ΔNDVI negativo)
- Clasificación de áreas según patrones de cambio
- Análisis de tendencias espaciales

💾 **Archivos Generados:**
- `data/processed/indices_YYYY.tif` - Índices espectrales para cada año
- `data/processed/estadisticas_indices.csv` - Tabla de estadísticas
- `data/processed/comparacion_indices.png` - Mapa comparativo
- `data/processed/histogramas_indices.png` - Distribuciones
- `data/processed/evolucion_temporal.png` - Gráficos de tendencias

## 8. Exportar Imágenes NDVI por Año para el Dashboard

Generamos imágenes PNG del NDVI para cada año, necesarias para la sección de comparación visual del dashboard de Streamlit.

In [ ]:
# Crear directorio de figuras si no existe
FIGURES_DIR = BASE_DIR / 'outputs' / 'figures'
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print("🖼️  Generando imágenes NDVI por año para el dashboard...")
print("="*70)

# Procesar cada año
for year in YEARS:
    print(f"\n📅 Procesando año {year}...")
    
    # Ruta del archivo de índices
    indices_file = OUTPUT_DIR / f'indices_{year}.tif'
    
    if not indices_file.exists():
        print(f"   ⚠️  Archivo no encontrado: {indices_file.name}")
        continue
    
    try:
        # Leer banda NDVI (banda 1)
        with rasterio.open(indices_file) as src:
            ndvi = src.read(1)
            profile = src.profile
            bounds = src.bounds
        
        # Crear figura
        fig, ax = plt.subplots(figsize=(14, 10))
        
        # Visualizar NDVI
        im = ax.imshow(ndvi, cmap='RdYlGn', vmin=-0.2, vmax=0.8,
                      extent=[bounds.left, bounds.right, bounds.bottom, bounds.top])
        
        # Configurar visualización
        ax.set_title(f'Índice de Vegetación (NDVI) - Peñaflor {year}', 
                    fontsize=18, fontweight='bold', pad=20)
        ax.set_xlabel('Longitud', fontsize=14)
        ax.set_ylabel('Latitud', fontsize=14)
        
        # Barra de color
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_label('NDVI', fontsize=14, fontweight='bold')
        cbar.ax.tick_params(labelsize=11)
        
        # Grid
        ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
        
        # Leyenda interpretativa
        legend_text = (
            'Valores NDVI:\n'
            '< 0.0: Agua/Suelo desnudo\n'
            '0.0-0.3: Vegetación escasa\n'
            '0.3-0.6: Vegetación moderada\n'
            '> 0.6: Vegetación densa'
        )
        ax.text(0.02, 0.98, legend_text, transform=ax.transAxes,
               fontsize=11, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.85,
                        edgecolor='black', linewidth=1))
        
        # Información del área
        area_text = f'Comuna de Peñaflor\n~202 km²'
        ax.text(0.98, 0.02, area_text, transform=ax.transAxes,
               fontsize=10, horizontalalignment='right',
               bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
        
        plt.tight_layout()
        
        # Guardar imagen
        output_file = FIGURES_DIR / f'ndvi_{year}.png'
        plt.savefig(output_file, dpi=150, bbox_inches='tight', facecolor='white')
        plt.close()
        
        # Calcular estadísticas
        ndvi_valid = ndvi[(ndvi >= -1) & (ndvi <= 1)]
        ndvi_mean = np.mean(ndvi_valid)
        ndvi_std = np.std(ndvi_valid)
        veg_area_pct = np.sum(ndvi_valid > 0.3) / len(ndvi_valid) * 100
        
        print(f"   ✓ Imagen guardada: {output_file.name}")
        print(f"   📊 NDVI medio: {ndvi_mean:.3f} ± {ndvi_std:.3f}")
        print(f"   🌳 Área con vegetación (NDVI>0.3): {veg_area_pct:.2f}%")
        
    except Exception as e:
        print(f"   ❌ Error al procesar {year}: {e}")
        continue

print("\n" + "="*70)
print("✅ Imágenes NDVI generadas exitosamente")
print(f"📁 Ubicación: {FIGURES_DIR}")
print("\n💡 Estas imágenes ahora aparecerán en el dashboard de Streamlit")
print("   en la sección '🔍 Comparación Visual Antes/Después'")
print("="*70)